In [ ]:
# Lab 5: Gradio UI + 최종 통합 (40분) - 실습 버전
# 소방청 민원처리 AI Agent 교육

"""
학습 목표:
- Gradio를 활용한 웹 UI 구축
- Lab 1-4의 모든 기능을 UI에 통합
- 실전 민원 처리 시스템 완성
"""

# ============================================================
# 환경 설정
# ============================================================

import sys, os
if 'google.colab' in sys.modules:
    !pip install openai chromadb pdfplumber gradio -q

import getpass
from openai import OpenAI
import pdfplumber
import chromadb
from chromadb.config import Settings
import gradio as gr
from typing import List, Dict
from datetime import datetime

# API 키 설정
print("[API 키 입력] OpenAI API 키를 입력하세요")
api_key = getpass.getpass("API Key: ")
client = OpenAI(api_key=api_key)

print("[완료] 환경 설정 완료!\n")

# ============================================================
# PDF 파일 업로드
# ============================================================

from google.colab import files

print("[파일 업로드] 소방기본법.pdf (10페이지)와 화재예방.pdf (44페이지)를 업로드하세요")
uploaded = files.upload()

for filename in uploaded.keys():
    print(f"[완료] {filename} 업로드 완료")

# ============================================================
# Lab 1-4 함수 통합 (빠른 재구축)
# ============================================================

print("\n[진행중] 시스템 초기화 중...")

# 기본 함수들
def classify_complaint(text: str) -> Dict[str, str]:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """소방 민원을 분류하고 다음 형식으로 답변하세요:

카테고리: [화재신고/소방시설/법규위반/일반문의]
긴급도: [높음/보통/낮음]
키워드: [핵심 키워드 3개]
요약: [한 줄 요약]"""
            },
            {"role": "user", "content": text}
        ],
        temperature=0.3,
        max_tokens=200
    )
    return {"원문": text, "분류_결과": response.choices[0].message.content}

def extract_text_from_pdf(pdf_path: str, max_pages: int = None) -> List[Dict]:
    pages_data = []
    with pdfplumber.open(pdf_path) as pdf:
        total_pages = len(pdf.pages) if max_pages is None else min(max_pages, len(pdf.pages))
        for i in range(total_pages):
            text = pdf.pages[i].extract_text()
            pages_data.append({"page_num": i + 1, "text": text, "char_count": len(text)})
    return pages_data

def chunk_text(pages_data: List[Dict], chunk_size: int = 500, overlap: int = 50) -> List[Dict]:
    chunks = []
    chunk_id = 0
    for page_data in pages_data:
        text = page_data['text']
        page_num = page_data['page_num']
        start = 0
        while start < len(text):
            end = start + chunk_size
            chunk_text = text[start:end]
            if chunk_text.strip():
                chunks.append({
                    "id": f"chunk_{chunk_id}",
                    "text": chunk_text,
                    "page": page_num,
                    "start_pos": start
                })
                chunk_id += 1
            start = end - overlap
    return chunks

def get_embedding(text: str) -> List[float]:
    response = client.embeddings.create(model="text-embedding-3-small", input=text)
    return response.data[0].embedding

# Vector DB 구축
print("[진행중] Vector DB 구축 중... (1-2분 소요)")
chroma_client = chromadb.Client(Settings(
    persist_directory="./chroma_db",
    anonymized_telemetry=False
))

try:
    chroma_client.delete_collection("fire_law")
    chroma_client.delete_collection("fire_prevention")
except:
    pass

# 소방기본법
fire_law_pages = extract_text_from_pdf("소방기본법.pdf", max_pages=10)
fire_law_chunks = chunk_text(fire_law_pages)
fire_law_collection = chroma_client.create_collection("fire_law")
for chunk in fire_law_chunks:
    embedding = get_embedding(chunk['text'])
    fire_law_collection.add(
        ids=[f"law_{chunk['id']}"],
        embeddings=[embedding],
        documents=[chunk['text']],
        metadatas=[{"page": chunk['page'], "source": "소방기본법"}]
    )

# 화재예방
prevention_pages = extract_text_from_pdf("화재예방.pdf")
prevention_chunks = chunk_text(prevention_pages)
prevention_collection = chroma_client.create_collection("fire_prevention")
for chunk in prevention_chunks:
    embedding = get_embedding(chunk['text'])
    prevention_collection.add(
        ids=[f"prev_{chunk['id']}"],
        embeddings=[embedding],
        documents=[chunk['text']],
        metadatas=[{"page": chunk['page'], "source": "화재예방가이드"}]
    )

# 검색 함수
def search_fire_law(query: str, n_results: int = 3) -> List[Dict]:
    query_embedding = get_embedding(query)
    results = fire_law_collection.query(query_embeddings=[query_embedding], n_results=n_results)
    return [
        {"text": results['documents'][0][i], "page": results['metadatas'][0][i]['page'], "source": results['metadatas'][0][i]['source']}
        for i in range(len(results['documents'][0]))
    ]

def search_fire_prevention(query: str, n_results: int = 3) -> List[Dict]:
    query_embedding = get_embedding(query)
    results = prevention_collection.query(query_embeddings=[query_embedding], n_results=n_results)
    return [
        {"text": results['documents'][0][i], "page": results['metadatas'][0][i]['page'], "source": results['metadatas'][0][i]['source']}
        for i in range(len(results['documents'][0]))
    ]

def search_both_sources(query: str, n_results_per_source: int = 2) -> Dict:
    return {
        "법령": search_fire_law(query, n_results_per_source),
        "예방가이드": search_fire_prevention(query, n_results_per_source),
        "total_count": n_results_per_source * 2
    }

# 보고서 생성
def generate_complaint_report(complaint_text: str) -> Dict:
    classification = classify_complaint(complaint_text)
    search_results = search_both_sources(complaint_text, n_results_per_source=2)
    
    law_context = "\n".join([f"- {r['source']} {r['page']}p: {r['text'][:200]}" for r in search_results['법령']])
    prevention_context = "\n".join([f"- {r['source']} {r['page']}p: {r['text'][:200]}" for r in search_results['예방가이드']])
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """당신은 소방청 민원 처리 담당자입니다.
다음 형식으로 공식 보고서를 작성하세요:

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
【민원 처리 보고서】
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. 민원 내용
   [민원 요약]

2. 분류 결과
   [카테고리, 긴급도, 키워드]

3. 관련 법령 및 근거
   [소방기본법 관련 내용]

4. 처리 방안
   [구체적인 대응 방법]

5. 예방 가이드
   [화재예방 참고사항]

6. 담당 부서 및 처리 기한
   [추천 담당 부서 및 기한]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
작성일시: [현재 날짜]
작성자: 소방청 AI Agent 담당자
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━"""
            },
            {
                "role": "user",
                "content": f"""다음 민원에 대한 처리 보고서를 작성하세요.

[민원 내용]
{complaint_text}

[분류 정보]
{classification['분류_결과']}

[참고 - 소방기본법]
{law_context}

[참고 - 화재예방 가이드]
{prevention_context}"""
            }
        ],
        temperature=0.4,
        max_tokens=1000
    )
    
    return {
        "민원원문": complaint_text,
        "분류": classification,
        "법령검색": search_results['법령'],
        "예방검색": search_results['예방가이드'],
        "보고서": response.choices[0].message.content
    }

print("[완료] 시스템 준비 완료!\n")

# ============================================================
# 실습 1: Gradio UI 함수
# ============================================================

print("="*60)
print("[실습 1] Gradio UI 함수 구현")
print("="*60)

def process_complaint_ui(complaint_text: str) -> str:
    """
    Gradio UI용 래퍼 함수
    
    TODO: 입력 검증과 오류 처리를 추가하세요
    
    입력: 민원 내용 (텍스트)
    출력: 처리 보고서 (텍스트)
    
    작업 순서:
    1. 입력 검증
       - 빈 문자열 체크
       - 최소 길이 체크 (10자 이상)
    2. try-except로 오류 처리
    3. generate_complaint_report() 호출
    4. 보고서 반환
    """
    
    # TODO: 입력 검증 추가
    # if not complaint_text or not complaint_text.strip():
    #     return "[경고] 민원 내용을 입력해주세요."
    # if len(complaint_text) < 10:
    #     return "[경고] 민원 내용이 너무 짧습니다..."
    
    # TODO: try-except 오류 처리 추가
    # try:
    #     result = generate_complaint_report(complaint_text)
    #     return result['보고서']
    # except Exception as e:
    #     return f"[오류 발생] {str(e)}\n\n해결 방법: ..."
    
    # 임시 구현 (위 TODO 완성 후 이 줄 삭제)
    return "[아직 구현되지 않았습니다]"

print("\n[완료] UI 함수 구현 완료")

# ============================================================
# 실습 2: Gradio 인터페이스 구축
# ============================================================

print("\n" + "="*60)
print("[실습 2] Gradio 인터페이스 구축")
print("="*60)

# 예시 민원들
examples = [
    ["아파트 복도에 소화기가 없는데 괜찮은가요?"],
    ["식당을 운영하는데 소방시설 점검은 언제 받아야 하나요?"],
    ["소방차 전용구역에 주차했다가 과태료를 받았습니다."],
    ["회사 사무실 소방시설 정기 점검 주기가 궁금합니다."],
    ["주택에 화재경보기 설치가 의무인가요?"]
]

"""
TODO: Gradio 인터페이스 생성하기

힌트:
interface = gr.Interface(
    fn=process_complaint_ui,  # 실행할 함수
    inputs=gr.Textbox(
        label="민원 내용",
        placeholder="민원을 입력하세요...",
        lines=7
    ),
    outputs=gr.Textbox(
        label="처리 보고서",
        lines=25
    ),
    title="소방청 민원처리 AI Agent",
    description="민원 내용을 입력하면 자동으로 처리 보고서를 생성합니다",
    examples=examples,
    theme=gr.themes.Soft()
)

주요 파라미터:
- fn: UI에서 실행할 함수
- inputs: 입력 컴포넌트 (Textbox, Number, Slider 등)
- outputs: 출력 컴포넌트
- title: UI 제목
- description: 설명
- examples: 예시 리스트
- theme: UI 테마
"""

# TODO: 여기에 Gradio 인터페이스 코드 작성
interface = None  # gr.Interface(...)로 변경

if interface:
    print("\n[완료] Gradio 인터페이스 구축 완료")
else:
    print("\n[아직 구현되지 않았습니다]")

# ============================================================
# 실습 3: UI 고급 기능 추가 (선택)
# ============================================================

print("\n\n" + "="*60)
print("[실습 3] UI 고급 기능 (선택 실습)")
print("="*60)

print("""
고급 기능 추가 아이디어:

1. 진행 상황 표시
   - gr.Progress() 사용
   - "1/3 민원 분류 중..." 메시지 표시

2. 다중 출력
   - outputs=[gr.Textbox(), gr.JSON()]
   - 보고서 + 검색 결과 JSON 동시 표시

3. 파일 다운로드
   - 보고서를 .txt 파일로 다운로드
   - gr.File() 컴포넌트 사용

4. 히스토리 기능
   - gr.State()로 이전 민원 저장
   - 민원 히스토리 표시

5. 설정 옵션
   - Temperature 조절 슬라이더
   - 검색 결과 개수 조절

예시 코드 (다중 출력):
def advanced_ui(complaint_text):
    result = generate_complaint_report(complaint_text)
    return result['보고서'], result  # 보고서, 전체 결과

interface = gr.Interface(
    fn=advanced_ui,
    inputs=gr.Textbox(...),
    outputs=[
        gr.Textbox(label="보고서"),
        gr.JSON(label="상세 정보")
    ],
    ...
)
""")

# ============================================================
# 실습 4: UI 실행 및 테스트
# ============================================================

print("\n\n" + "="*60)
print("[실습 4] Gradio UI 실행")
print("="*60)

print("""
[안내] Gradio UI를 실행합니다.

실행 방법:
1. interface.launch(share=True) 실행
2. 자동으로 public URL이 생성됩니다
3. URL을 클릭하여 웹 인터페이스를 엽니다
4. 민원을 입력하고 테스트합니다

Colab 환경 특징:
- share=True: 외부 접속 가능한 링크 생성
- 링크는 72시간 동안 유효
- 다른 사람들과 공유 가능
- debug=True: 에러 메시지 상세 표시
""")

# TODO: UI 실행하기
if interface:
    print("\n[실행] Gradio UI 시작...")
    # interface.launch(share=True, debug=True)
    print("[참고] 위 주석을 해제하여 실행하세요")
else:
    print("[먼저 실습 2에서 interface를 생성해야 합니다]")

# ============================================================
# 최종 정리
# ============================================================

print("\n" + "="*60)
print("[완료] Lab 5 완료!")
print("="*60)
print("""
학습 내용:
1. Gradio 기본 사용법
2. UI 함수 구현 (입력 검증, 오류 처리)
3. 인터페이스 구성 (입력/출력/예시)
4. 웹 UI 배포 (share=True)

최종 시스템:
- 민원 입력 → 자동 분류 → 법령 검색 → 보고서 생성
- 웹 인터페이스로 누구나 쉽게 사용 가능
- 외부 공유 가능한 링크 생성

실전 활용:
1. 부서 내 공유하여 실무에 활용
2. 다양한 민원으로 테스트하여 정확도 검증
3. 필요시 프롬프트 수정하여 맞춤화

전체 Lab 완료 축하합니다!
━━━━━━━━━━━━━━━━━━━━━━━━━━
Lab 1: OpenAI API 기초
Lab 2: RAG 기초 (PDF + 임베딩)
Lab 3: 고급 검색 (다중 문서)
Lab 4: 보고서 자동 생성
Lab 5: Gradio UI 완성 ← 현재
━━━━━━━━━━━━━━━━━━━━━━━━━━

이제 실전 민원을 처리해보세요!

실습 체크리스트:
[ ] 실습 1: process_complaint_ui 함수 완성 (입력 검증, 오류 처리)
[ ] 실습 2: Gradio 인터페이스 생성
[ ] 실습 3: (선택) 고급 기능 추가
[ ] 실습 4: UI 실행 및 테스트
""")